In [13]:
#This script is used to process the dynamic few shot learning dataset from the vehicle dataset

import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

DATA_PATH = "../../Datasets/Vehice dataset/Car details v3.csv.xls"
TEST_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test.csv"
TRAIN_PATH = "../../Datasets/Vehice dataset/Downsampled/Train/train.csv"
VALID_PATH = "../../Datasets/Vehice dataset/Downsampled/Valid/valid.csv"
TEST_SAMPLED_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled.csv"
OUTPUT_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled_few_shot.csv"




In [14]:
def find_nearest_neighbors(df, target_row, K=3):

    vect = TfidfVectorizer()
    transformed = vect.fit_transform(df['combined'])

    new_sentence_vec = vect.transform([target_row])

    cosine_similarities = cosine_similarity(new_sentence_vec, transformed).flatten()

    nearest_neighbor_ind = np.argsort(cosine_similarities)[-K:]
    nearest_neighbors = df.iloc[nearest_neighbor_ind]
    similarity_scores = cosine_similarities[nearest_neighbor_ind]


    return_df = nearest_neighbors.copy()
    return_df['og_ind'] = nearest_neighbor_ind
    return_df['Similarity_Score'] = similarity_scores
    return_df.reset_index(drop=True, inplace=True)


    examples_sentences = [] 
    for i in range(K):
        example_sentence = """Features:
name: {name},
year: {year},
km_driven: {km_driven},
fuel: {fuel},
seller_type: {seller_type},
transmission: {transmission},
owner: {owner},
mileage: {mileage},
engine: {engine},
max_power: {max_power},
torque: {torque},
seats: {seats}

Output: "price": {selling_price} INR """.format(
            name=return_df.iloc[i]['name'],
            year=return_df.iloc[i]['year'],
            km_driven=return_df.iloc[i]['km_driven'],
            fuel=return_df.iloc[i]['fuel'],
            seller_type=return_df.iloc[i]['seller_type'],
            transmission=return_df.iloc[i]['transmission'],
            owner=return_df.iloc[i]['owner'],
            mileage=return_df.iloc[i]['mileage'],
            engine=return_df.iloc[i]['engine'],
            max_power=return_df.iloc[i]['max_power'],
            torque=return_df.iloc[i]['torque'],
            seats=return_df.iloc[i]['seats'], 
            selling_price=return_df.iloc[i]['selling_price']
        )
        examples_sentences.append(example_sentence)

    return return_df, examples_sentences

In [15]:
#Get Ramdon Sample of Dataframe
df_samples = pd.read_csv(TEST_SAMPLED_PATH)
train_df = pd.read_csv(TRAIN_PATH)
df_samples = df_samples.drop(columns=['prediction', 'prompt'])

train_df['combined'] = train_df.apply(lambda row: ' '.join(str(row[col]) for col in train_df.columns), axis=1)
df_samples['combined'] = df_samples.apply(lambda row: ' '.join(str(row[col]) for col in df_samples.columns), axis=1)  
    

In [16]:
# Add columns to store the example sentences
for i in range(3):
    df_samples['example_prompt_{}'.format(i+1)] = None
    df_samples['example_score_{}'.format(i+1)] = None
    df_samples['example_index_{}'.format(i+1)] = None

for index in range(len(df_samples)):
    print(index)
    target_row = df_samples['combined'][index]
    return_df, example_sentences = find_nearest_neighbors(train_df, target_row)
    for i in range(len(return_df)):
        df_samples.loc[index, 'example_prompt_{}'.format(i+1)] = example_sentences[i]
        df_samples.loc[index, 'example_score_{}'.format(i+1)] = return_df['Similarity_Score'][i]
        df_samples.loc[index, 'example_index_{}'.format(i+1)] = return_df['og_ind'][i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
print(df_samples['example_prompt_1'][0])

Features:
name: Ford Figo Aspire 1.5 TDCi Trend,
year: 2017,
km_driven: 35000,
fuel: Diesel,
seller_type: Individual,
transmission: Manual,
owner: First Owner,
mileage: 25.83 kmpl,
engine: 1498 CC,
max_power: 99 bhp,
torque: 215Nm@ 1750-3000rpm,
seats: 5.0

Output: "price": 700000 INR 


In [18]:
# Save the dataframe
df_samples = df_samples.drop(columns=['combined'])
df_samples.to_csv(OUTPUT_PATH, index=False)
   